In [469]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.cluster import silhouette_score

In [470]:
DEBUG = False

# read in both datasets
reading_history_df   = pd.read_csv("data/db/reading_history_database.csv")
assessment_scores_df = pd.read_csv("data/db/screening_assessment_scores.csv")
print("reading history matrix original dimensions:   ", reading_history_df.shape)
print("assessment scores matrix original dimensions: ", assessment_scores_df.shape)

reading history matrix original dimensions:    (455, 34)
assessment scores matrix original dimensions:  (453, 26)


# Preprocess reading_history database

In [471]:
# ----- shorten the column names for readability
shortened_cols = ["Q"+str(q) for q in range(1,34)]
shortened_cols.insert(0, "Participant")
reading_history_df.columns = shortened_cols

In [472]:
# ------ delete irrelevant columns 
del reading_history_df['Q30'] # comments entered by subjects
del reading_history_df['Q32'] # comments entered by subjects
del reading_history_df['Q33'] # level of education

In [473]:
# ------- remove rows containing all NULL values
reading_history_df = reading_history_df.dropna(axis=0, how='all')

In [474]:
# ------- find and remove any duplicate keys ('Participant')    #df[~df.name.isin(value_list)]
rd_h = reading_history_df['Participant'].value_counts()
reading_history_duplicates = rd_h[rd_h > 1]
if DEBUG: print("\nreading_history duplicate keys:")
if DEBUG: print(reading_history_duplicates.sort_index())

duplicate_participants_to_remove = reading_history_duplicates.index.tolist()
reading_history_df = reading_history_df[~reading_history_df.Participant.isin(duplicate_participants_to_remove)]

In [475]:
# ------- remove row that has 'Participant' number as "INCOMPLETE"
reading_history_df = reading_history_df[~reading_history_df.Participant.isin(['INCOMPLETE'])]

In [476]:
# ------- Q26 cleanup (fill in NULL values with most frequently occurring)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q26: ", reading_history_df.Q26.isnull().sum())
if DEBUG: print("Frequency of values in Q26: \n", reading_history_df['Q26'].value_counts())

most_frequent_value_in_Q26 = reading_history_df['Q26'].value_counts().index[0]
reading_history_df.Q26.fillna(most_frequent_value_in_Q26, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q26: ", reading_history_df.Q26.isnull().sum())
if DEBUG: print("Frequency of values in Q26: \n", reading_history_df['Q26'].value_counts())

In [477]:
# ------- Q27 cleanup (fill in NULL values with most frequently occurring)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q27: ", reading_history_df.Q27.isnull().sum())
if DEBUG: print("Frequency of values in Q27: \n", reading_history_df['Q27'].value_counts())

most_frequent_value_in_Q27 = reading_history_df['Q27'].value_counts().index[0]
reading_history_df.Q27.fillna(most_frequent_value_in_Q27, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q27: ", reading_history_df.Q27.isnull().sum())
if DEBUG: print("Frequency of values in Q27: \n", reading_history_df['Q27'].value_counts())

In [478]:
# ------- Q28 cleanup (fill in NULL values)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q28: ", reading_history_df.Q28.isnull().sum())
if DEBUG: print("Frequency of values in Q28: \n", reading_history_df['Q28'].value_counts())
    
# replace 'INCOMPLETE' with NaN
reading_history_df['Q28'] = reading_history_df['Q28'].apply(pd.to_numeric, errors='coerce')
    
most_frequent_value_in_Q28 = reading_history_df['Q28'].value_counts().index[0]
reading_history_df.Q28.fillna(most_frequent_value_in_Q28, inplace=True)

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q28: ", reading_history_df.Q28.isnull().sum())
if DEBUG: print("Frequency of values in Q28: \n", reading_history_df['Q28'].value_counts())

In [479]:
# ------- Q29 cleanup (make all lowercase and strip whitespace)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q29: ", reading_history_df.Q29.isnull().sum())
if DEBUG: print("Frequency of values in Q29: \n", reading_history_df['Q29'].value_counts())

# convert to lowercase and strip leading and trailing whitespace
reading_history_df['Q29'] = reading_history_df['Q29'].apply(lambda x: x.lower().strip())

# replace spaces with underscores
reading_history_df['Q29'] = reading_history_df['Q29'].apply(lambda x: '_'.join(x.split(" "))) 

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q29: ", reading_history_df.Q29.isnull().sum())
if DEBUG: print("Frequency of values in Q29: \n", reading_history_df['Q29'].value_counts())

In [480]:
# ------- Q31 cleanup (make all lowercase and strip whitespace)
if DEBUG: print("BEFORE:\nNumber of NULL values in Q31: ", reading_history_df.Q31.isnull().sum())
if DEBUG: print("Frequency of values in Q31: \n", reading_history_df['Q31'].value_counts())

# fill in NULL value with most frequent
most_frequent_value_in_Q31 = reading_history_df['Q31'].value_counts().index[0]
reading_history_df.Q31.fillna(most_frequent_value_in_Q31, inplace=True)

# convert to lowercase and strip leading and trailing whitespace
reading_history_df['Q31'] = reading_history_df['Q31'].apply(lambda x: x.lower().strip())

# consolidate all 'no' responses
def process_no_responses(x):
    if x == 'no (n/a)' or x == 'no, only child':
        return('no')
    else:
        return(x)
reading_history_df['Q31'] = reading_history_df['Q31'].apply(process_no_responses) 

# replace whitespaces with underscores
reading_history_df['Q31'] = reading_history_df['Q31'].apply(lambda x: '_'.join(x.split(" ")))

if DEBUG: print("\nAFTER:\nNumber of NULL values in Q31: ", reading_history_df.Q31.isnull().sum())
if DEBUG: print("Frequency of values in Q31: \n", reading_history_df['Q31'].value_counts())

In [481]:
# ------- Q33 cleanup
#if DEBUG: print("Number of NULL values in Q33: ", reading_history_df.Q33.isnull().sum())
#if DEBUG: print("\nFrequency of values in Q33: \n", reading_history_df['Q33'].value_counts())
    
# fill in NULL value with most frequent
#most_frequent_value_in_Q33 = reading_history_df['Q33'].value_counts().index[0]
#reading_history_df.Q33.fillna(most_frequent_value_in_Q33, inplace=True)

# lower case and strip whitespace
#reading_history_df['Q33'] = reading_history_df['Q33'].apply(lambda x: x.lower().strip())

# consolidate responses for level of education

#if DEBUG: print("Number of NULL values in Q33: ", reading_history_df.Q33.isnull().sum())
#if DEBUG: print("\nFrequency of values in Q33: \n", reading_history_df['Q33'].value_counts())

In [482]:
# ----- one-hot encoding of categories (Q29, Q31)
Q29_dummies = pd.get_dummies(reading_history_df.Q29, prefix='Q29').astype(int)
reading_history_df = pd.concat([reading_history_df,Q29_dummies],axis=1) 
reading_history_df.drop('Q29',inplace=True,axis=1)

Q31_dummies = pd.get_dummies(reading_history_df.Q31, prefix='Q31').astype(int)
reading_history_df = pd.concat([reading_history_df,Q31_dummies],axis=1) 
reading_history_df.drop('Q31',inplace=True,axis=1)

In [483]:
# ---- check final pre-processed reading_history dataframe
print("pre-processed reading_history dataframe dimensions: ", reading_history_df.shape)
print("\nNULL values: ", reading_history_df.isnull().sum().sum())

reading_history_df.to_csv('reading_history_pre_processed.csv')
#reading_history_df.head(5)

pre-processed reading_history dataframe dimensions:  (433, 35)

NULL values:  0


# Preprocess assessment_scores

In [484]:
# assessment_scores_df.head()

In [485]:
# --- update column names for readability

# strip leading and trailing whitepsace, add underscores, rename 'Participant Number'
new_columns = pd.Series(assessment_scores_df.columns).apply(lambda x: x.strip())
new_columns = new_columns.apply(lambda x: '_'.join(x.split(" ")))
new_columns = new_columns.tolist()
new_columns[0] = 'Participant'

assessment_scores_df.columns = new_columns
#assessment_scores_df.head()

In [486]:
# ------- remove rows containing all NULL values
assessment_scores_df = assessment_scores_df.dropna(axis=0, how='all')

In [487]:
# ------- find and remove any rows with duplicate keys ('Participant')    #df[~df.name.isin(value_list)]
if DEBUG: print("BEFORE:")
if DEBUG: print("assessment scores dimensions: ", assessment_scores_df.shape)
    
a_s = assessment_scores_df['Participant'].value_counts()
assessment_scores_duplicates = a_s[a_s > 1]

if DEBUG: print("assessment_scores duplicate keys:")
if DEBUG: print(assessment_scores_duplicates.sort_index())

duplicate_participants_to_remove = assessment_scores_duplicates.index.tolist()
assessment_scores_df = assessment_scores_df[~assessment_scores_df.Participant.isin(duplicate_participants_to_remove)]
if DEBUG: print("\nAFTER:")
if DEBUG: print("assessment scores dimensions: ", assessment_scores_df.shape)


In [488]:
# ----- delete row marked "INCOMPLETE"
if DEBUG: print("BEFORE: ", assessment_scores_df.shape)
assessment_scores_df = assessment_scores_df[assessment_scores_df.Participant != 'INCOMPLETE']
if DEBUG: print("AFTER: ", assessment_scores_df.shape)

In [489]:
# ---- replace all strings (e.g., "Skipped", "Incomplete", etc. with NaN) except for Participant
if DEBUG: print("BEFORE: Total NULL values: ", assessment_scores_df.isnull().sum().sum())
filter_col = assessment_scores_df.columns.tolist()
filter_col.remove('Participant')

assessment_scores_df[filter_col] = assessment_scores_df[filter_col].apply(pd.to_numeric, errors='coerce')
if DEBUG: print("AFTER: Total NULL values: ", assessment_scores_df.isnull().sum().sum())

In [490]:
# ---- fill in NULL values (with median value)
if DEBUG: print("BEFORE:\nNULL values: \n", assessment_scores_df.isnull().sum())
    
median_per_column = assessment_scores_df[filter_col].apply(lambda x: x.median(),axis=0)
if DEBUG: print("\nMedian values per column:\n", median_per_column)
    
assessment_scores_df[filter_col] = assessment_scores_df[filter_col].fillna(median_per_column,axis=0)

# ---- fill in NULL values (with median value)
if DEBUG: print("\nAFTER:\nNULL values: \n", assessment_scores_df.isnull().sum())


In [491]:
# ---- check final pre-processed reading_history dataframe
if DEBUG: print("\nNULL values: \n", assessment_scores_df.isnull().sum())
print("pre-processed assessment_scores dataframe dimensions: ", assessment_scores_df.shape)

assessment_scores_df.to_csv('assessment_scores_pre_processed.csv')
# assessment_scores_df.head(3)

pre-processed assessment_scores dataframe dimensions:  (428, 26)


# Merge 2 databases

In [492]:
# merge 2 databases on 'Participant' as key into one (inner join -> intersection of keys)
merged_history_and_scores_df = pd.merge(reading_history_df, assessment_scores_df, how='inner', on='Participant')

In [493]:
# reset 'Participant' as the index
merged_history_and_scores_df.set_index('Participant', inplace=True)
print("merged history and scores dimensions: ", merged_history_and_scores_df.shape)
if DEBUG: print(merged_history_and_scores_df.dtypes)

merged history and scores dimensions:  (421, 59)


In [494]:
# TODO feature scaling

In [495]:
# K-means on unscaled data
km = KMeans(n_clusters=4)
km.fit(merged_history_and_scores_df)
merged_history_and_scores_df['kmeans_3'] = [ "cluster_" + str(label) for label in km.labels_ ]
merged_history_and_scores_df['kmeans_3'].value_counts()

cluster_0    194
cluster_2    105
cluster_1    103
cluster_3     19
Name: kmeans_3, dtype: int64